# TP1 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

## Problema 2 - Cálculo do SVP

Na criptografia pós-quântica os reticulados inteiros (“hard lattices”) e os problemas a eles associados são uma componente essencial. Um reticulado inteiro pode ser definido por uma matriz $L \in \mathbb{Z}^{m\times n}$ (com $m > n$) de inteiros e por um inteiro primo $q \geq 3$.

Como 'inputs', o problema recebe:
1. Número natural $n$;
2. Número natural $m$, tal que $|m| > |n| + 1$;
3. Número natural primo $q$, tal que $|q| > |m|$.
4. Os elementos $L_{j,i}$ são gerados aleatória e uniformemente no intervalo inteiro $\{-d, \cdots, d\}$ sendo  $d = (q-1)/2$.

O chamado problema do vetor curto (SVP) consiste no cálculo de um vetor de inteiros: $e\in \{-1,0,1\}^m$
não nulo que verifique a seguinte relação matricial:
$$\forall_{i < n}, \quad (\sum_{j< m} e_j \times \mathsf{L}_{j,i}) \equiv 0 \mod q$$


### Análise

Para resolver o problema do SVP, decidimos usar duas famílias de variáveis, uma binária e outra inteira.

A família de variáveis binárias $e_{j,l}$ é definida com a seguinte semântica, para $l \in \{0,1,2\}$ e $0 <= j < m$:

$$e_{j,0} == 1 \quad \mbox{se e só se} \quad e[j] == -1$$
$$e_{j,1} == 1 \quad \mbox{se e só se} \quad e[j] == 0$$
$$e_{j,2} == 1 \quad \mbox{se e só se} \quad e[j] == 1$$

A família de variáveis inteiras $k_{i}$, para $i < n$ servirá para conseguir iterar pelos números inteiros, de modo a se encontrar os múltiplos de $q$ para cada resultado da multiplicação entre $e$ e $L$.

O problema dispõe de restrições que deverão ser tratadas:

1. Para cada posição $j < m$ de $e$, $e[j]$ só admite um único valor;
2. Pelo menos um elemento de $e$ tem de ser não nulo;
3. Multiplicar o vetor $e$ pela matriz $L$ deve resultar num vetor de múltiplos de $q$.

Do mesmo modo, o problema admite a seguinte optimização:

1. Maximizar o número de componentes nulas de $e$.

## Implementação

Para a resolução do problema, utilizaremos a biblioteca de programação linear do $OR-Tools$, o $pywraplp$. Portanto, começamos por instalar o OR-Tools, importar a biblioteca e inicializar o $solver$, que chamaremos de 'solver'.

In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

Como exemplo introdutório, daremos os seguintes valores aos 'inputs' do problema.

In [ ]:
n = 3
m = 8
q = 17

Para a família de variáveis binárias $e_{j,l}$, construíremos uma matriz de alocação $e$ de valores em $\{0,1\}^{M \times 3}$, para representar as variáveis binárias. Para a família de variáveis inteiras $k_{i}$ declaramos um vetor $k \in \mathbb{Z}^{n}$ Declaramos ambos como dicionários:

In [ ]:
e = {}
k = {}
for j in range(m):
    e[j] = [solver.BoolVar('e[%i][-1]' % j),
            solver.BoolVar('e[%i][0]' % j),
            solver.BoolVar('e[%i][1]' % j)]
for i in range(n):
    k[i] = solver.IntVar(-solver.infinity(), solver.infinity(), 'k[%i]' % i)

### Restrições

Nesta seccção, apresentamos as implementações das restrições ao problema.

1. Para cada posição $j < m$ de $e$, $e[j]$ só admite um único valor;

$$ \forall_{j<m}, \quad e_{j,0} + e_{j,1} + e_{j,2} == 1 $$

In [ ]:
for j in range(m):
    solver.Add(e[j][0] + e[j][1] + e[j][2] == 1)

2. Pelo menos um elemento de $e$ tem de ser não nulo;

Deste modo, o sumatório das variáveis $e_{j,1}$ não deve igualar o máximo de posições do vetor $e$, $m$.

$$ \sum_{j<m} e_{j,1} <= m-1 $$

In [ ]:
solver.Add(sum([e[j][1] for j in range(m)]) <= m-1)

3. Multiplicar o vetor $e$ pela matriz $L$ deve resultar num vetor de múltiplos de $q$.

Para cada valor de $j<m$, no máximo uma das variáveis $e_{j,0}$ e $e_{j,2}$ é igual a $1$, representando $-1$ e $1$, respetivamente. Desse modo:

$$ \forall_{j<m}, \quad e[j] == (e_{j,2} - e_{j,0}) $$

$$ \forall_{i<n}, \quad (\sum_{j<m} (e_{j,2} - e_{j,0}) * L[j][i])  == q * k[i]$$

In [ ]:
for i in range(n):
    solver.Add((sum([(e[j][2] - e[j][0])*L[j][i] for j in range(m)])) == q * k[i])

### Optimização

O critério de optimização para este problma é o seguinte:

1. Maximizar o número de componentes nulas de $e$.

$$ max (\sum_{j<m} e_{j,1}) $$

In [ ]:
solver.Maximize(sum([e[j][1] for j in range(m)]))

Por fim, tenta-se calcular uma solução:

In [ ]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print(L)
    print("")
    for i in range(n):
        print(k[i].solution_value(), end = " ")
    print("")
    for j in range(m):
        for k in range(3):
            if e[j][k].solution_value():
                print(k-1, end = " ")
else:
    print(L)
    print("Não é possível resolver o problema")

## Exemplos

Nesta seccção, experimentamos com valores progressivamente maiores de $n$, o que necessariamente aumenta a complexidade do problema, tal como, na nossa experiência, a duração e a probabilidade de não haver solução.

De modo a facilitar a testagem dos exemplos, na célula colapsada seguinte encontra-se a função $SVP(m,n,q)$. Esta função compõe todos os passos da implementação.

In [ ]:
from ortools.linear_solver import pywraplp
import numpy

#  n  > 30, impossível
# |m| > 1 + |n|
# |q| > |m|

def SVP(m,n,q):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    L = numpy.random.randint(-(q-1)/2, (q-1)/2, (m, n))
    e = {}
    k = {}
    #e é um vetor de tamanho m com valores inteiros -1, 0 ou 1
    for j in range(m):
        e[j] = [solver.BoolVar('e[%i][-1]' % j),
                solver.BoolVar('e[%i][0]' % j),
                solver.BoolVar('e[%i][1]' % j)]
        #1. e só pode ter um elemento por cada j
        solver.Add(e[j][0] + e[j][1] + e[j][2] == 1)
    
            
    #k é um vetor de tamanho n com valores inteiros
    for i in range(n):
        k[i] = solver.IntVar(-solver.infinity(), solver.infinity(), 'k[%i]' % i)
    
    #2. Tem de existir um elemento não nulo em e
    solver.Add(sum([e[j][1] for j in range(m)]) <= m-1)
    
    #3. Multiplicar e por L deve resultar num vetor de múltiplos de q
    for i in range(n):
        solver.Add((sum([(e[j][2] - e[j][0])*L[j][i] for j in range(m)])) == q * k[i])
    
    solver.Maximize(sum([e[j][1] for j in range(m)]))
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print(L)
        print("")
        for i in range(n):
            print(k[i].solution_value(), end = " ")
        print("")
        for j in range(m):
            for k in range(3):
                if e[j][k].solution_value():
                    print(k-1, end = " ")
        return True
    else:
        print(L)
        print("Não é possível resolver o problema")
        return False

[[ -5  12  -1  -8  14  15]
 [ -4 -13  -5 -11  -1   4]
 [ 10 -12  -1 -18 -13 -15]
 [ -6 -18 -12  -7 -13  -8]
 [  4   1 -17 -16  -2  -5]
 [-15  -8 -13  11 -10   9]
 [ 17   6  -5   9  14   1]
 [ -5 -13   5   6   1  10]
 [  5 -11  -6   4  16  -1]
 [  1  -2 -15 -15   7  -8]
 [-11 -17 -10   8  15  -9]
 [ 17   0   8   7   2   7]
 [ 17  -4 -14   3   6  -8]
 [-15   4  16  -3  11   9]
 [ 17  -2  -3   2 -16   6]
 [ 15  -2   4  -6  12   7]]
Não é possível resolver o problema
[[ -8  -5  -2 -14 -16  -6]
 [  0   2   8  -8   2   3]
 [ 13  -5  -5  -7 -18  12]
 [ 16  -6  10   0 -13  12]
 [ 14  -1 -14   5 -18 -11]
 [ -2  -8  16  -5   1   5]
 [ 11 -15   7 -13  14   5]
 [  4 -14 -12   7 -17   5]
 [ -9  15   7 -11 -16 -15]
 [-12   8  14 -18 -11   0]
 [-11 -14   8  -2  11 -15]
 [-17  -6   0 -10  -5 -18]
 [ -9 -16   5   5   5  -6]
 [ -1   3  -9  -6   4 -10]
 [ 13 -12  -9   8 -10  -6]
 [ 17   4   7  -5   3  13]]
Não é possível resolver o problema
[[ -2   7   5   5 -15 -12]
 [  7 -17  -1   3  11   6]
 [  6 -14 

### Exemplo 1

In [ ]:
SVP(1, 4, 11)

### Exemplo 2 

In [ ]:
SVP(3, 8, 11)

### Exemplo 3

In [ ]:
SVP(5, 16, 37)

### Exemplo 4

In [ ]:
SVP(7, 16, 37)

###### 